In [1]:
import pandas as pd
import numpy as np
from OT2_code import OT2Directions as OT2D
from OT2_code import Create_Directions as CD
import opentrons.simulate as simulate
from datetime import datetime
from pytz import timezone 

## Import values for Volume, Time, and Order
The three arrays contain information on what to perform. 
- **v_array**: This is the volume array and it contains information on the volumes of each stock solution, in microliters, used to make one sample. Each column represents a stock solution and each row represents a sample. 
- **o_array**: This is the order in which the stock solutions are added. Each row represents a sample and each column represents the next stock solution that should be added. The number 1 represents the first stock solution that should be added. 
- **t_array**: This is the time array and it contains information on the delay times before adding each stock solution. The first colum represents the time delay that occurs before adding the second stock solution to the first. Note that the delay times are applied to the stock solutions of the samples after they are ordered according to the values in the o_array. The units of the time delay are in units of the **action_time** which is the amount of time it takes in seconds to perform one action. More information on this can be found in the next section. 

In [2]:
v_array = pd.read_csv('Data/221021/Volume.csv').values
t_array = pd.read_csv('Data/221021/Time.csv').values
o_array = pd.read_csv('Data/221021/Order.csv').values

## Create the Direction Array
The function CD.create_directions will transform the volume, time, and order arrays into a single array with the directions that the OT2 can follow. The OT2 will perform one action (pick up tip - transfer stock to destination - drop tip) per row and every action will take the same amount of time which can be set by the keyword argument **action_time**. The first column of the direction_array variable contains information on the volume that needs to be transferred. The second column contains information on the sample number. The third column contains information on the stock number to use. A row that contains negative numbers means that the OT2 will not perform any action for a duration equal to the absolute value of the negative number multiplied by the time it takes to perform one action. Any sample with a volume = 0 and time_delay = 0, will be completely ignored. 

In [4]:
direction_array = CD.create_directions(v_array, t_array, o_array)

Set the time that each action should take with the keyword argument **action_time**, which has units of seconds. 

In [5]:
exp = OT2D.experiment()
protocol = exp.simulate()
exp.calculate_exp_duration(direction_array, action_time = 60)

/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults
/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults


Experiment will take a total time of:
3 hours
22 minutes


## Start the OT2
The following line of code will determine where to place the stock solutions in the labware specified in the file *protocol.csv*. The variable **vial_volume** is the maximum volume of each stock solution vial in mircoliters. It is assumed that every stock solution vial has the same volume.  

In [6]:
vial_volume = 17000
direction_array = exp.stock_solution_ranges(direction_array, vial_volume)

Stock 0 on  A1 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 1 on  A2 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 2 on  A3 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 3 on  A4 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 4 on  B1 of 20mLscintillation 12 Well Plate 18000 µL on 1


Determine which well in the wellplate to start in

In [7]:
start_location = 0

The following line of code will simulate the OT2 commands used to perform the experiment. 

In [8]:
protocol = exp.simulate()
exp.perform_directions(protocol, direction_array, start_location = start_location)

/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults
/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults


Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Aspirating 78.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 500.0 uL/sec
Air gap
Aspirating 20.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 500.0 uL/sec
Dispensing 78.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at A1 of Corning 96 Well Plate 360 µL Flat on 6
Blowing out at A1 of Corning 96 Well Plate 360 µL Flat on 6
Dropping tip into A1 of Opentrons 96 Tip Rack 300 µL on 7
Delaying for 0 minutes and 59.992 seconds
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Aspirating 31.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 500.0 uL/sec
Air gap
Aspirating 20.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 500.0 uL/sec
Dispensing 31.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at A3 of Corning 96 Well Plate 360 µL Flat on 6
Blowing out at A3 of Corning 96 Well Plate 360

### Run the Experiment

In [11]:
Pacific = timezone('US/Pacific')
p_time = datetime.now(Pacific)
print(p_time.strftime('Start Time: %H:%M:%S'))

Start Time: 22:14:48


In [ ]:
protocol = exp.execute()
exp.perform_directions(protocol, direction_array, start_location = start_location)

In [12]:
Pacific = timezone('US/Pacific')
p_time = datetime.now(Pacific)
print(p_time.strftime('End Time: %H:%M:%S'))

End Time: 22:14:55


## Test Accuracy of Experiment

In [9]:
exp.test(exp.exp_data, v_array, t_array, o_array)

Time: The average time delay error is 1.85e+03 seconds
Order: Check Passed, Actual Order is equal to specified Order
Volume: Check Passed, Actual Volume is equal to specified Volume
